
### 1\. Введение в массивы

#### Что такое массивы в PostgreSQL?

Представьте, что вам нужно хранить для каждого товара список тегов (например, "электроника", "смартфон", "новинка"). Вместо того чтобы создавать отдельную таблицу `product_tags` и связывать её с товарами, вы можете создать в таблице `products` одну колонку типа `text[]` (массив строк) и хранить все теги прямо в ней.

**Преимущества:**

  * **Упрощение схемы данных:** Сокращается количество таблиц и связей (JOIN'ов), что делает запросы проще и часто быстрее для чтения.
  * **Атомарность данных:** Связанные элементы (как теги одного товара) хранятся вместе.
  * **Интуитивность:** Для некоторых задач, вроде хранения телефонных номеров пользователя или размеров одежды, массив выглядит более естественной структурой.

**Основные характеристики:**

  * **Однородность:** Все элементы в одном массиве должны быть одного типа данных (например, `integer[]`, `text[]`, `date[]`).
  * **Многомерность:** Массивы могут быть многомерными, как матрицы. Например, `integer[][]` для хранения игрового поля.
  * **Гибкий размер:** Размер массива не фиксирован и может меняться.

#### Примеры реальных сценариев использования

  * **E-commerce:** Хранение списка размеров или цветов для товара, тегов, ID похожих товаров.
  * **Социальные сети:** Список ID друзей пользователя, ID лайкнувших пост, список интересов.
  * **Геоданные:** Координаты маршрута в виде массива точек.
  * **Настройки:** Хранение пользовательских настроек в виде массива ключ-значение.

-----

### 2\. Инициализация и срезы

#### Синтаксис создания массивов

Массивы можно создавать несколькими способами.

  * **Литералы массива:** Используются фигурные скобки `{}` и строки в одинарных кавычках.

    ```sql
    SELECT '{apple, banana, cherry}'::text[];
    ```

  * **Конструктор `ARRAY[]`:** Более гибкий и предпочтительный способ, особенно когда элементы массива являются результатом другого запроса.

    ```sql
    SELECT ARRAY['apple', 'banana', 'cherry'];
    ```

  * **Создание колонки типа массив:** При создании или изменении таблицы указывается тип данных с квадратными скобками.

Давайте добавим в нашу базу **northwind** колонку для хранения альтернативных контактных телефонов сотрудников.

```sql
ALTER TABLE employees
ADD COLUMN alternative_phones TEXT[];

-- Заполним эту колонку для одного из сотрудников
UPDATE employees
SET alternative_phones = ARRAY['(206)555-9857', '(206)555-9482']
WHERE first_name = 'Nancy' AND last_name = 'Davolio';
```

#### Доступ к элементам

Важнейшая особенность PostgreSQL — **индексация массивов начинается с 1**, а не с 0, как в большинстве языков программирования\!

  * **Индексация:** `array[index]`

    ```sql
    -- Получим первый альтернативный телефон Нэнси Даволио
    SELECT alternative_phones[1]
    FROM employees
    WHERE first_name = 'Nancy';
    -- Результат: (206)555-9857
    ```

  * **Срезы `[start:end]`:** Позволяют получить часть массива.

    ```sql
    -- Представим, что у сотрудника много телефонов, и мы хотим получить со 2-го по 3-й
    UPDATE employees
    SET alternative_phones = '{"(206)555-1111", "(206)555-2222", "(206)555-3333", "(206)555-4444"}'
    WHERE employee_id = 1;

    SELECT alternative_phones[2:3]
    FROM employees
    WHERE employee_id = 1;
    -- Результат: {"(206)555-2222", "(206)555-3333"}
    ```

  * **Работа с многомерными массивами:** Индексы указываются через запятую `array[row][column]`.

    ```sql
    -- Пример шахматной доски 3x3
    SELECT ('{{R,N,B},{P,P,P},{ , , }}'::text[])[1][2];
    -- Результат: N (конь)
    ```

-----

### 3\. Массивы и операторы

PostgreSQL предлагает богатый набор операторов для работы с массивами.

  * **Конкатенация `||`:** Объединяет два массива или добавляет элемент в массив.

    ```sql
    -- Добавим еще один номер телефона существующему списку
    UPDATE employees
    SET alternative_phones = alternative_phones || '(206)555-5555'::text
                                         -- или || ARRAY['(206)555-5555']
    WHERE employee_id = 1;

    SELECT alternative_phones FROM employees WHERE employee_id = 1;
    -- Результат: {"(206)555-1111", "(206)555-2222", "(206)555-3333", "(206)555-4444", "(206)555-5555"}
    ```

  * **Операторы сравнения (`=`, `<>`, etc.):** Сравнивают массивы целиком. Они должны быть идентичны (порядок важен\!).

  * **Поиск элементов:** Это одна из самых мощных возможностей.

      * **`ANY()` (или `= ANY()`)**: Проверяет, равен ли элемент **хотя бы одному** из элементов массива.
      * **`ALL()`**: Проверяет, удовлетворяет ли элемент условию **для всех** элементов массива.
      * **`@>` (содержит)**: Проверяет, содержит ли левый массив **все** элементы правого массива.
      * **`<@` (содержится в)**: Проверяет, содержатся ли **все** элементы левого массива в правом.
      * **`&&` (пересекается)**: Проверяет, есть ли у массивов **хотя бы один** общий элемент.

Давайте усложним пример. Добавим в таблицу `products` колонку с ID категорий поставщиков, у которых можно купить аналогичный товар.

```sql
-- Добавляем колонку и заполняем ее для нескольких товаров
ALTER TABLE products ADD COLUMN similar_supplier_ids INTEGER[];

UPDATE products SET similar_supplier_ids = ARRAY[2, 3] WHERE product_id = 1; -- У поставщиков 2 и 3 есть аналоги "Chai"
UPDATE products SET similar_supplier_ids = ARRAY[18, 19] WHERE product_id = 17; -- Аналоги "Alice Mutton"
UPDATE products SET similar_supplier_ids = ARRAY[3, 7, 18] WHERE product_id = 34; -- Аналоги "Sasquatch Ale"
```

**Примеры запросов:**

```sql
-- 1. Найти все товары, аналоги которых поставляет поставщик с ID = 3
-- Используем ANY, т.к. нам нужно найти `3` внутри массива
SELECT product_name, similar_supplier_ids
FROM products
WHERE 3 = ANY(similar_supplier_ids);
```

```sql
-- 2. Найти все товары, аналоги которых есть и у поставщика 3, и у поставщика 18
-- Используем оператор "содержит" @>
SELECT product_name, similar_supplier_ids
FROM products
WHERE similar_supplier_ids @> ARRAY[3, 18];
```

```sql
-- 3. Найти все товары, аналоги которых есть хотя бы у одного из поставщиков 18 или 19
-- Используем оператор "пересечение" &&
SELECT product_name, similar_supplier_ids
FROM products
WHERE similar_supplier_ids && ARRAY[18, 19];
```

-----

### 4\. `VARIADIC` и `FOREACH`

Эти конструкции используются внутри функций на языке PL/pgSQL для более сложной логики.

#### Функции с `VARIADIC` параметрами

`VARIADIC` позволяет передать в функцию переменное число аргументов, которые внутри функции "собираются" в массив.

  * **Синтаксис:** `function_name(VARIADIC arr_name an_array_type)`

**Пример:** Создадим функцию, которая будет находить все заказы, содержащие **любой** из переданных ID товаров.

```sql
CREATE OR REPLACE FUNCTION GetOrdersByProductIDs(VARIADIC p_product_ids integer[])
RETURNS TABLE (order_id smallint, order_date date, product_id smallint) AS $$
BEGIN
    RETURN QUERY
    SELECT o.order_id, o.order_date, od.product_id
    FROM orders o
    JOIN order_details od ON o.order_id = od.order_id
    WHERE od.product_id = ANY(p_product_ids)
    ORDER BY o.order_id, od.product_id;
END;
$$ LANGUAGE plpgsql;

-- Вызываем функцию с разным числом аргументов
SELECT * FROM GetOrdersByProductIDs(11, 42, 72); -- 3 продукта
SELECT * FROM GetOrdersByProductIDs(1); -- 1 продукт
```

#### Цикл `FOREACH`

`FOREACH` позволяет удобно перебирать элементы массива внутри PL/pgSQL функции.

  * **Синтаксис:** `FOREACH var IN ARRAY array_expression LOOP ... END LOOP;`

**Пример:** Напишем функцию, которая принимает массив ID сотрудников и повышает зарплату (гипотетическую, т.к. в `northwind` нет зарплаты) каждому на 10%, но не более чем на 20% для тех, кто из Лондона.

```sql
-- Сначала добавим гипотетическую колонку salary
ALTER TABLE employees ADD COLUMN salary NUMERIC(10, 2);
UPDATE employees SET salary = 2000 WHERE city = 'London';
UPDATE employees SET salary = 1800 WHERE city <> 'London';

-- Теперь сама функция
CREATE OR REPLACE FUNCTION BulkRaiseSalary(employee_ids integer[])
RETURNS TABLE (employee_id int, old_salary numeric, new_salary numeric) AS $$
DECLARE
    emp_id int;
    current_city text;
    current_salary numeric;
BEGIN
    -- Перебираем каждый ID из входного массива
    FOREACH emp_id IN ARRAY employee_ids
    LOOP
        -- Получаем текущие данные сотрудника
        SELECT city, salary INTO current_city, current_salary
        FROM employees WHERE employees.employee_id = emp_id;

        -- Логика повышения
        IF current_city = 'London' THEN
            new_salary := current_salary * 1.05; -- +5% для лондонцев
        ELSE
            new_salary := current_salary * 1.10; -- +10% для остальных
        END IF;

        -- Обновляем данные в таблице
        UPDATE employees SET salary = new_salary WHERE employees.employee_id = emp_id;

        -- Возвращаем результат для отчета
        RETURN QUERY SELECT emp_id, current_salary, new_salary;
    END LOOP;
END;
$$ LANGUAGE plpgsql;

-- Вызываем функцию для сотрудников 1, 3 и 5
SELECT * FROM BulkRaiseSalary(ARRAY[1, 3, 5]);
```
----